# Lab 05: Column Generation

Made & Presented by Bo Tang

In this lab, we will implement the **Column Generation**, which is commonly used to solve large-scale linear programming problems with <u>an enormous number of variables</u>, such as the Cutting Stock Problem. The goal is to solve the master problem by iteratively adding new columns (variables) based on subproblem solutions.

In [3]:
pip install gurobipy

In [4]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

### Column Generation Review

Column Generation is often used to solve large-scale linear programming problems where the number of decision variables is <u>extremely large</u>, making it intractable to enumerate and include all variables in the model initially.

This method is particularly useful in problems like cutting stock, vehicle routing, and crew scheduling, where only a small subset of the decision variables (columns) are expected to be active in the optimal solution.

The key idea behind Column Generation is to start with a smaller, manageable subset of decision variables (columns) in the **Master Problem** and iteratively add new, more promising variables by solving a **Subproblem**. The Subproblem identifies which variables (columns) have the potential to improve the solution of the Master Problem by evaluating their **reduced cost**.

Consider a large-scale linear program in the following form, where we want to minimize the objective function:

$$
\max_x \mathbf{c}^\top \mathbf{x}
$$
Subject to
$$
\mathbf{A} \mathbf{x} \leq \mathbf{b}
$$
$$
\mathbf{x} \geq \mathbf{0}
$$

Here, $\mathbf{x}$ represents the decision variables, $\mathbf{c}$ the cost coefficients, and $\mathbf{A} \mathbf{x} \leq \mathbf{b}$ represents the constraints.

#### Master Problem

In Column Generation, we start by solving a restricted version of this problem, known as the **Restricted Master Problem (RMP)**, which only includes a subset of decision variables $\{ x_1, x_2, \cdots, x_k \}$ and the corresponding cosntraint columns $\{ \mathbf{a}_1, \mathbf{a}_2, \cdots, \mathbf{a}_k \}$.

Thus, the Master is just the same as the large model formulation, but using the variables found in the previous $k$ iterations.

$$
\max_x \sum_{j=1}^k c_j x_j
$$
Subject to
$$
\sum_{j=1}^k \mathbf{a}_j x_j \leq \mathbf{b}
$$
$$
x_j \geq 0 \quad \forall j
$$

#### Subproblem

The Subproblem identifies new columns (variables) to be added to the Master Problem by solving a pricing problem. The goal is to find the column with the most negative reduced cost (if minimizing), which is computed as:
$$
\text{Reduced cost} = c_j - \boldsymbol{\lambda}^\top \mathbf{a}_j
$$
Where:
- $c_j$ is the cost coefficient of the new column,
- $\mathbf{a}_j$ is the column vector of the new variable,
- $\mathbf{\lambda}$ is the vector of dual variables (shadow prices) obtained from solving the Master Problem.

In order to find the variable which has **the most postive reduced cost**, we solve the following price problem:
$$
\min_{\mathbf{a}_j} \boldsymbol{\lambda}^\top \mathbf{a}_j - c_j
$$
Subject to the **constraints specific to the problem**.

If the reduced cost of the new column is negative, this column is added to the Master Problem. The process then repeats: the Master Problem is solved again with the new column, and the Subproblem is solved to search for additional columns. This iterative process continues until no more columns with negative reduced cost can be found, indicating that the optimal solution has been reached.

**Question:**
- What happens in a minimization master?

#### Integer Linear Problem

Unfortunately, CG does not solve ILP problems, so we proceed as follows:

1. Relax the integer variables and solve the relaxed problem using Column Generation as a linear program (LP).
2. Reintroduce the integer variables to the master problem to obtain a solution.

It is important to note that after relaxing the integer constraints and reintroducing them, Column Generation CANNOT guarantee global optimality for integer linear problems.

##### Branch & Price (Optional)

**Branch & Price** is an algorithm that combines Column Generation with the **Branch-and-Bound** method, specifically designed to handle complex problems with integer variables. In this approach, Column Generation is embedded within the Branch-and-Bound framework to solve the linear relaxation of each node.

In Branch & Price, the algorithm first uses Column Generation to solve the relaxed linear problem, then introduces branching operations at each node of the search tree to handle the integer variables. Unlike traditional Branch-and-Bound, Branch & Price only generates columns relevant to the current branch, which makes it more efficient for large-scale problems.

### Example 1: Cutting Stock Problem in Notes

You sell copper wire to the retail market in lengths of 3', 5', and 9'. The respective demand for these lengths is 25, 20, and 15 units. You buy copper wire on the wholesale market in 107' lengths. The objective is to determine how many 107' lengths to buy and how to cut them to satisfy the demand with minimal cost.

In [5]:
# available lengths
lengths = [3, 5, 9]
# demand
demand = [25, 20, 15]

#### Exact Large Model Formulation

We can model this problem as an integer linear programming (ILP) problem.

Let $x_j$  be the number of times cutting pattern $j$ is used, where each pattern represents a possible way of cutting the 107' wholesale length into lengths of 3', 5', and 9'.

The optimization model aims to minimize the total number of wholesale copper wire lengths used:

$$
\min_x \sum_{j=1}^k x_j
$$
Subject to
$$
\sum_{j=1}^k a_{3j} x_j \geq 25
$$
$$
\sum_{j=1}^k a_{5j} x_j \geq 20
$$
$$
\sum_{j=1}^k a_{9j} x_j \geq 15
$$
$$
x_j \geq 0 \quad \forall j
$$

**Question:**
- How many cutting patterns do we have?

In [6]:
import numpy as np

# wholesale length of copper wire
wholesale_length = 107

# generate all combinations of cutting patterns
def generate_combinations():
    combinations = []
    for a3 in range(wholesale_length // lengths[0] + 1):
        for a5 in range(wholesale_length // lengths[1] + 1):
            for a9 in range(wholesale_length // lengths[2] + 1):
                if (a3 * lengths[0] + a5 * lengths[1] + a9 * lengths[2]) <= wholesale_length:
                    combinations.append((a3, a5, a9))
    return combinations

# generate all possible cutting patterns
patterns = generate_combinations()

# display the number of patterns and a sample
print("Total number of valid cutting patterns:", len(patterns))
print("Sample patterns:", patterns[:10])  # Show first 10 patterns

Total number of valid cutting patterns: 1921
Sample patterns: [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 0, 5), (0, 0, 6), (0, 0, 7), (0, 0, 8), (0, 0, 9)]


We can solve this ILP problem.

In [7]:
# create Gurobi model
m = gp.Model("cutting_stock")

# decision variables, one for each cutting pattern
x = m.addVars(len(patterns), vtype=GRB.INTEGER, name="x")

# objective: Minimize the total number of wires used
m.setObjective(gp.quicksum(x[i] for i in range(len(patterns))), GRB.MINIMIZE)

# demand constraints for each length 3 5 9
m.addConstr(gp.quicksum(patterns[i][0] * x[i] for i in range(len(patterns))) >= demand[0], "Demand_3ft")
m.addConstr(gp.quicksum(patterns[i][1] * x[i] for i in range(len(patterns))) >= demand[1], "Demand_5ft")
m.addConstr(gp.quicksum(patterns[i][2] * x[i] for i in range(len(patterns))) >= demand[2], "Demand_9ft")

# optimize the model
m.optimize()

# display the results
if m.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Minimum number of wires needed: {m.objVal}\n")
    print("Cutting patterns used:")
    for i in range(len(patterns)):
        if x[i].x > 0:
            print(f"Pattern {patterns[i]}: used {x[i].x} times")
else:
    print("No optimal solution found.")

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 1921 columns and 4970 nonzeros
Model fingerprint: 0x21c94254
Variable types: 0 continuous, 1921 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 35.0000000
Presolve removed 0 rows and 68 columns
Presolve time: 0.02s
Presolved: 3 rows, 1853 columns, 4830 nonzeros
Found heuristic solution: objective 3.0000000
Variable types: 0 continuous, 1853 integer (2 binary)

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 

### Column Generation

We will model this using a Column Generation approach where:
- The Master Problem determines how to use the cuts.
- The Subproblem generates new cutting patterns (columns) based on the current solution of the Master.

#### Initial Pattern

We will start with a simple initial cutting pattern:
$$
\begin{pmatrix}
1 \\
1 \\
11
\end{pmatrix}
$$

In [22]:
init_pattern = [1, 1, 11] #initial patter for master

#### Master Problem

Fill the code for master problem.

In [23]:
len(init_pattern)

3

In [28]:
for i in range(len(init_pattern)):
    print(init_pattern[0])

1
1
1


In [31]:
import gurobipy as gp
from gurobipy import GRB

def solve_master_problem(patterns):
    # create Gurobi model
    m = gp.Model("master")
    # turn off log
    m.Params.outputFlag = 0

    # Decision variables: number of times each pattern is used
    x = m.addVars(len(patterns), name="x", vtype=GRB.CONTINUOUS)

    # Objective: Minimize the number of total copper wires used
    m.setObjective(gp.quicksum(x[i] for i in range(len(patterns))), GRB.MINIMIZE)

    # Constraints: Meet demand for each length (3', 5', 9')
    m.addConstr(gp.quicksum(patterns[i][0] * x[i] for i in range(len(patterns))) >= 25, name="Demand_3ft")
    m.addConstr(gp.quicksum(patterns[i][1] * x[i] for i in range(len(patterns))) >= 20, name="Demand_5ft")
    m.addConstr(gp.quicksum(patterns[i][2] * x[i] for i in range(len(patterns))) >= 15, name="Demand_9ft")

    # Optimize the master problem
    m.optimize()

    # Get the objective value
    objval = m.objVal

    # Return the dual variables (shadow prices) from the demand constraints
    λ = [m.getConstrByName("Demand_3ft").Pi,
         m.getConstrByName("Demand_5ft").Pi,
         m.getConstrByName("Demand_9ft").Pi]

    x = [x[i].x for i in range(len(patterns))]

    return objval, λ

# Example of calling the function with an initial pattern
init_pattern = [(1, 1, 11)]  # This is your initial simple cut pattern from the screenshot
objval, λ = solve_master_problem(init_pattern)

print("Objective value:", objval)
print("Dual variables (shadow prices):", λ)





Objective value: 25.0
Dual variables (shadow prices): [1.0, 0.0, 0.0]


In [ ]:
init_pattern[0]

(1, 1, 11)

#### Subproblem

Next, we solve the Subproblem to find new cutting patterns that could improve the solution. The Subproblem will find a cutting pattern with the most negative reduced cost.

In [33]:
def solve_subproblem(λ):
    # create the subproblem to find new cutting patterns
    m = gp.Model("Subproblem")
    # turn off log
    m.Params.outputFlag = 0

    # Decision variables: number of 3', 5', and 9' lengths in the new pattern
    a3 = m.addVar(vtype=GRB.INTEGER, name="a3")  # number of 3' lengths
    a5 = m.addVar(vtype=GRB.INTEGER, name="a5")  # number of 5' lengths
    a9 = m.addVar(vtype=GRB.INTEGER, name="a9")  # number of 9' lengths

    # Objective: Maximize (λ^T * a - 1)
    m.setObjective(λ[0] * a3 + λ[1] * a5 + λ[2] * a9 - 1, GRB.MAXIMIZE)

    # Constraint: Total length of cuts must be <= 107' (or the total available length)
    m.addConstr(3 * a3 + 5 * a5 + 9 * a9 <= 107, name="LengthConstraint")

    # Optimize the subproblem
    m.optimize()

    # Objective value (reduced cost)
    objval = m.objVal

    # Get the new cutting pattern found
    new_pattern = [a3.x, a5.x, a9.x]  # Values of decision variables for 3', 5', 9'

    return objval, new_pattern

# Example usage of the subproblem with given dual values (λ)
λ = [1.0, 0, 0]  # Example dual variables (shadow prices)
objval, new_pattern = solve_subproblem(λ)

print("\nObjective value:", objval)
print("New pattern:", new_pattern)



Objective value: 34.0
New pattern: [35.0, -0.0, -0.0]


#### Column Generation Iteration

We will now implement the Column Generation algorithm to iteratively solve the master and Subproblem. The algorithm will add new patterns until no further improvements can be made.

In [ ]:
init_pattern

[(1, 1, 11)]

In [34]:
def column_generation():
    # Step 1: Initialize with the given pattern
    patterns = [(1, 1, 11)]  # Start with the initial pattern

    # Step 2: Iterate until no new pattern with positive reduced cost is found
    while True:
        # Solve the master problem with the current patterns
        objval_master, λ = solve_master_problem(patterns)

        # Solve the subproblem using the dual variables from the master problem
        objval_subproblem, new_pattern = solve_subproblem(λ)

        print(f"Master Objective: {objval_master}")
        print(f"Subproblem Objective: {objval_subproblem}")
        print(f"New Pattern: {new_pattern}")

        # Step 3: Check if the reduced cost from the subproblem is non-positive
        if objval_subproblem <= 0:
            # Optimal solution found, stop the algorithm
            print("Optimal solution found!")
            break
        else:
            # Step 4: Add the new pattern to the list of patterns in the master problem
            patterns.append(new_pattern)

    # Final solution
    print(f"Final Patterns: {patterns}")
    print(f"Final Objective: {objval_master}")

    return patterns

# Example usage
patterns = column_generation()

# Display the number of patterns and a sample
print("Total number of valid cutting patterns:", len(patterns))
print("Patterns:", patterns)


Master Objective: 25.0
Subproblem Objective: 34.0
New Pattern: [35.0, -0.0, -0.0]
Master Objective: 20.142857142857142
Subproblem Objective: 19.4
New Pattern: [0.0, 21.0, -0.0]
Master Objective: 2.9264069264069263
Subproblem Objective: 0.01904761904761898
New Pattern: [34.0, 1.0, 0.0]
Master Objective: 2.9131652661064424
Subproblem Objective: 0.01680672268907557
New Pattern: [4.0, 19.0, -0.0]
Master Objective: 2.897196261682243
Subproblem Objective: -0.0
New Pattern: [34.0, 1.0, -0.0]
Optimal solution found!
Final Patterns: [(1, 1, 11), [35.0, -0.0, -0.0], [0.0, 21.0, -0.0], [34.0, 1.0, 0.0], [4.0, 19.0, -0.0]]
Final Objective: 2.897196261682243
Total number of valid cutting patterns: 5
Patterns: [(1, 1, 11), [35.0, -0.0, -0.0], [0.0, 21.0, -0.0], [34.0, 1.0, 0.0], [4.0, 19.0, -0.0]]


#### Solve ILP

Now reintroduce the integer variables to find a solution.

In [35]:
# solve ILP
m = gp.Model("cutting_stock")
# decision variables, one for each cutting pattern
x = m.addVars(len(patterns), vtype=GRB.INTEGER, name="x")
# objective: Minimize the total number of wires used
m.setObjective(gp.quicksum(x[i] for i in range(len(patterns))), GRB.MINIMIZE)
# demand constraints for each length 3 5 9
m.addConstr(gp.quicksum(patterns[i][0] * x[i] for i in range(len(patterns))) >=demand[0], "Demand_3ft")
m.addConstr(gp.quicksum(patterns[i][1] * x[i] for i in range(len(patterns))) >=demand[1], "Demand_5ft")
m.addConstr(gp.quicksum(patterns[i][2] * x[i] for i in range(len(patterns))) >=demand[2], "Demand_9ft")
# optimize the model
m.optimize()
# display the results
print("\nOptimal solution found:")
print(f"Minimum number of wires needed: {m.objVal}\n")
print("Cutting patterns used:")
for i in range(len(patterns)):
  if x[i].x > 0:
    print(f"Pattern {patterns[i]}: used {x[i].x} times")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 5 columns and 9 nonzeros
Model fingerprint: 0x9efcea72
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 8.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 4.000000e+00, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     

In [ ]:
# create Gurobi model
m = gp.Model("cutting_stock")

# decision variables, one for each cutting pattern
x = m.addVars(len(patterns), vtype=GRB.INTEGER, name="x")

# objective: Minimize the total number of wires used
m.setObjective(gp.quicksum(x[i] for i in range(len(patterns))), GRB.MINIMIZE)

# demand constraints for each length 3 5 9
m.addConstr(gp.quicksum(patterns[i][0] * x[i] for i in range(len(patterns))) >= demand[0], "Demand_3ft")
m.addConstr(gp.quicksum(patterns[i][1] * x[i] for i in range(len(patterns))) >= demand[1], "Demand_5ft")
m.addConstr(gp.quicksum(patterns[i][2] * x[i] for i in range(len(patterns))) >= demand[2], "Demand_9ft")

# optimize the model
m.optimize()

# display the results
print("\nOptimal solution found:")
print(f"Minimum number of wires needed: {m.objVal}\n")
print("Cutting patterns used:")
for i in range(len(patterns)):
    if x[i].x > 0:
        print(f"Pattern {patterns[i]}: used {x[i].x} times")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 5 columns and 9 nonzeros
Model fingerprint: 0x9efcea72
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 8.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 4.000000e+00, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     

We can observe that the solution obtained through Column Generation may not always be the optimal solution for integer linear problems.

### Example 2: Clothing Company Shipment

A clothing company is in the process of creating a set of new shipments $s \in S$. Each shipment combines a set of available clothing items from the set $F$. Each clothing item $i \in F$ has an expected sales revenue of $r_i$ dollars, handling cost of $c_i$, and the total number of units available of item $i$ is $d_i$. Also, the maximum number of distinct shipments that can be made is equal to $K$. Finally, for each shipment, the total handling cost should not exceed 50\% of the expected revenue.
You have been hired by this company to help optimize the shipments that should be sent out in order to maximize the total expected revenue.

The company has decided to model the problem in the following way:
Let $S$ be the set of all feasible shipment patterns that can be created.
Let $a_{is}$ be the number of clothing item $i$ in shipment $s$.
Let $r_s = \sum_{i \in F} a_{is} r_i$ be the expected revenue from shipment $s$.

Let variables $x_s$ be equal to 1 if shipment $s$ will be sent, and 0 otherwise.

Then the problem formulation follows:
$$
\max \sum_{s \in S} r_s x_s
$$
Subject to:
$$
\sum_{s \in S} a_{is} x_s \leq d_i \quad \forall i \in F
$$
$$
\sum_{s \in S} x_s \leq K
$$
$$
x_s \in \{0, 1\} \quad \forall s \in S
$$

The first set of constraints ensures that no more than the available amount is sent out for each item $i$.
The last constraint ensures that no more than $K$ shipments are sent.

Suppose you are given dual variables $\alpha_i$ for the first set of constraints, and $\beta$ for the last constraint.

Now consider solving the LP relaxation of this problem and applying column generation.

- Write down a formula for the reduced cost for a variable $x_s$.
- Assuming that you are using column generation for solving the problem, explain how you might set up your pricing problem.
- Explain what the decision variables will be and write down the objective function and constraints.